In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAILike
from agno.tools.tavily import TavilyTools
from agno.workflow import Workflow, Step
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown

load_dotenv()

model = OpenAILike(
    id="openai/gpt-5-mini",
    base_url=os.environ.get("OPENAI_BASE_URL"),
    api_key=os.environ.get("OPENAI_API_KEY"),
)

search_agent = Agent(
    model=model,
    name="DuckSearcher",
    tools=[TavilyTools(api_key="tvly-dev-")], 
    instructions=(
        "Ты — агент, использующий DuckDuckGo. "
        "Всегда предоставляй итоговый ответ строго на русском языке."
        "Если человек пишет вопрос на английском, всеравнво переводи все на русский и отвечай на русском."
    ),
    markdown=True,
    stream=False,
)

workflow_1 = Workflow(
    name="tavily_workflow",
    description="Поиск в интернете через Tavily",
    steps=[
        Step(
            name="search_step",
            agent=search_agent
        )
    ]
)
result = workflow_1.run("Кто такая Instasamka?")
if result and result.content:
    display(Markdown(result.content))

### Кратко
Instasamka — сценический псевдоним российской медийной личности и певицы Дарьи Евгеньевны Зотеевой (в последнее время также фигурирует фамилия Еропкина). Родилась 11 мая 2000 года в Тобольске. Как артистка и блогер известна под именем «Инстасамка» — от «Instagram» + «самка».

### Чем прославилась
- Начинала как инстаграм‑блогер (ранний псевдоним — «Президентша»), стала заметной в интернете примерно с 2017–2019 годов.  
- Впоследствии развила карьеру в музыке: рэп/поп‑реп, затем сместила вектор в сторону поп‑музыки.  
- Известна провокационным образом и эффектными хайповыми действиями в медиапространстве; активно работает в TikTok, Instagram и на YouTube.  
- Её треки и клипы регулярно обсуждаются в российской интернет‑медиасреде, а некоторые из релизов попадали в рейтинги профильных изданий.

### Формат творчества и репутация
- Жанр: поп‑рэп / поп.  
- Образ: эпатажный, вызывающий — это часть её сценической стратегии.  
- Вокруг неё были и остаются споры и скандалы, что только увеличивает медийность.

### Где смотреть и читать
Для подробной биографии и дискографии можно посмотреть её страницу в Википедии (ru/en), официальный сайт и её профили в социальных сетях (Instagram, TikTok, YouTube, стрим‑платформы).

Если хотите, могу кратко пересказать основные этапы карьеры или перечислить наиболее известные треки — скажите, что интересует.

In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAILike
from agno.tools.tavily import TavilyTools
from agno.tools.reasoning import ReasoningTools
from agno.workflow import Workflow, Step
from dotenv import load_dotenv
import os
import json
from IPython.display import display, Markdown

load_dotenv()
class SimpleKnowledge:
    def __init__(self, filepath="tmp/knowledge.json"):
        self.filepath = filepath
        os.makedirs(os.path.dirname(filepath), exist_ok=True)
        with open(self.filepath, 'w', encoding='utf-8') as f:
            json.dump({}, f)
        self.data = {}
    
    def add(self, key, value):
        self.data[key] = value
        with open(self.filepath, 'w', encoding='utf-8') as f:
            json.dump(self.data, f, ensure_ascii=False, indent=2)
    
    def get_all(self):
        return "\n".join([f"- {k}: {v}" for k, v in self.data.items()])

knowledge = SimpleKnowledge()
knowledge.add("Инстасамка_секрет", "Секретный факт: Любимое блюдо Инстасамки — блины Маникена")
knowledge.add("Инстасамка_био", "Инстасамка (Дарья Зотеева) — российская поп-исполнительница, блогер. ""Родилась 2 июля 2000 года. Известна песнями 'За деньги да', 'Popstar'.")
knowledge.add("Инстасамка_карьера", "Начала карьеру в TikTok в 2020 году. Выпустила дебютный альбом 'Сама' в 2023 году. " "Работала с продюсером Nikolay Gubankov.")



model = OpenAILike(
    id="openai/gpt-5-mini",
    supports_native_structured_outputs=True,
    base_url=os.environ.get("OPENAI_BASE_URL"),
    api_key=os.environ.get("OPENAI_API_KEY"),
    reasoning_effort="high",
)

smart_agent = Agent(
    model=model,
    name="SmartResearcher",
    tools=[TavilyTools(api_key="tvly-dev-"), ReasoningTools()],
    instructions=(
        "Ты интеллектуальный ии агент, с доступом к базе данных и интернету \n"
        f"!!! ВНУТРЕННЯЯ БАЗА ЗНАНИЙ (Приоритет №1) !!!:\n{knowledge.get_all()}\n"
        "СНАЧАЛА проверь базу знаний. Если ответ там — используй его.\n"
        "Если нет — ищи в DuckDuckGo.\n"
        "Проверяй несколько источников, чтобы точно дать правильный ответ."
        "Синтезируй один единый ответ из всего найденного."
        "Если нужно посчитать/подумать — используй Reasoning.\n"
        "Отвечай на русском языке."
        "ВАЖНО: \n"
        "Давай краткие и структурированные ответы не более 250 слов"
        "Если находишь противоречивую информацию, укажи на это в ответе"
    ),
    markdown=True,
    stream=False,
    reasoning=True,          
    num_history_messages=10, 
)

workflow_2 = Workflow(
    name="smart_workflow",
    steps=[Step(name="search_step", agent=smart_agent)]
)

def run_test(text, session_id):
    print(f"\n--- ВОПРОС: {text} ---")
    try:
        res = workflow_2.run(text, session_id=session_id)
        if res and res.content:
            display(Markdown(res.content))
    except Exception as e:
        print(f"Ошибка: {e}")

session_id = "main_session_v1"
run_test("Какое любимое блюдо у Инстасамки согласно секретным данным?", session_id)
run_test("Кто такая INSTASAMKA? Как её зовут на самом деле?", session_id)
run_test("Как зовут её мужа?", session_id)
run_test("Посчитай разницу в возрасте между ней и её мужем.", session_id)


--- ВОПРОС: Какое любимое блюдо у Инстасамки согласно секретным данным? ---


INFO Reasoning model: OpenAILike is not a native reasoning model, defaulting to manual Chain-of-Thought reasoning

Согласно внутренней (секретной) базе данных: **любимое блюдо Инстасамки — блины Маникена**.

Примечание: в открытых источниках прямого подтверждения этой записи не найдено; упоминания в сети касаются других рецептов (хорба, пицца и т.п.), но не опровергают внутреннюю информацию.


--- ВОПРОС: Кто такая INSTASAMKA? Как её зовут на самом деле? ---


INFO Reasoning model: OpenAILike is not a native reasoning model, defaulting to manual Chain-of-Thought reasoning

...

INFO Reasoning model: OpenAILike is not a native reasoning model, defaulting to manual Chain-of-Thought reasoning

Её мужа зовут Олег Еропкин — в творчестве он известен как Moneyken.

Примечание: в источниках встречаются разные варианты написания фамилии (Еропкин/Еропхин/Yeropkin) и расхождения по году свадьбы, но имя Олег и псевдоним Moneyken подтверждаются несколькими независимыми источниками.


--- ВОПРОС: Посчитай разницу в возрасте между ней и её мужем. ---


INFO Reasoning model: OpenAILike is not a native reasoning model, defaulting to manual Chain-of-Thought reasoning

Краткий ответ:

- Даты рождения: Инстасамка (Дарья Зотеева) — 2 июля 2000 (внутренняя БД); муж — Moneyken (Олег Еропкин) — 20 сентября 2000 (веб‑источники, напр. Wikipedia).  
- Разница в возрасте: 2 месяца 18 дней (80 дней). Инстасамка старше.

Примечание: в некоторых внешних источниках указана дата рождения Инстасамки 11 мая 2000 — в этом случае разница была бы 4 месяца 9 дней (132 дня). В ответе использована внутренняя база как приоритет.

In [ ]:
from agno.agent import Agent
from agno.models.openai import OpenAILike
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.workflow import Workflow, Step, Parallel
from agno.workflow.types import StepInput, StepOutput
from pydantic import BaseModel
from dotenv import load_dotenv
import os
from IPython.display import display, Markdown

load_dotenv()

if 'knowledge' not in locals():
    class MockKnowledge:
        def get_all(self): return "Данные не загружены."
    knowledge = MockKnowledge()

model = OpenAILike(
    id="openai/gpt-5-mini",
    base_url=os.environ.get("OPENAI_BASE_URL"),
    api_key=os.environ.get("OPENAI_API_KEY"),
)


searcher = Agent(
    model=model,
    role="Searcher",
    tools=[TavilyTools(api_key="tvly-dev-")], 
    instructions=(
        "Ты ищешь информацию в интернете через DuckDuckGo.\n"
        "Найди 2-3 надёжных источника.\n"
        "Отвечай кратко (до 150 слов), на русском.\n"
        "Формат: '[ИНТЕРНЕТ] <факты>' + список источников."
    ),
    stream=False
)

keeper = Agent(
    model=model,
    role="Keeper",
    instructions=(
        f"Проверь внутреннюю базу:\n{knowledge.get_all()}\n"
        "Если нашёл — верни точно как в базе.\n"
        "Если не нашёл — верни '[БАЗА ПУСТА]'.\n"
        "Отвечай кратко, на русском."
    ),
    stream=False
)

writer = Agent(
    model=model,
    role="Writer",
    instructions=(
        "Ты синтезируешь финальный отчёт из данных других агентов.\n\n"
        "ПРАВИЛА ПРИОРИТИЗАЦИИ:\n"
        "1. Если в базе есть данные — используй их как ОСНОВУ.\n"
        "2. Если в интернете есть СВЕЖИЕ факты (дата, новости) — дополни ими базу.\n"
        "3. Если база и интернет ПРОТИВОРЕЧАТ:\n"
        "   - Укажи это явно: ' Противоречие:'\n"
        "   - Объясни разницу\n"
        "   - Рекомендуй проверить официальные источники\n"
        "4. Если оба агента вернули пустые данные — честно скажи об этом.\n\n"
        "ФОРМАТ (до 250 слов):\n"
        "**Итог:** <основной ответ>\n"
        "**Источники:** <база/интернет/оба>\n"
        "**Примечания:** <если есть противоречия>\n"
    ),
    stream=False
)


class ResearchTopic(BaseModel):
    topic: str

def synthesize(step_input: StepInput) -> StepOutput:
    topic = step_input.input.topic
    
    web_res = step_input.get_step_content("Gathering")["Web_Search"]
    db_res = step_input.get_step_content("Gathering")["Internal_Check"]
    
    prompt = (
        f"Тема: {topic}\n"
        f"Web: {web_res}\n"
        f"Internal DB: {db_res}\n"
        "Сформируй полный ответ."
    )
    
    response = writer.run(prompt)
    return StepOutput(content=response.content, success=True)

workflow_3 = Workflow(
    name="multi_agent_workflow",
    steps=[
        Parallel(
            Step(name="Web_Search", agent=searcher),
            Step(name="Internal_Check", agent=keeper),
            name="Gathering"
        ),
        Step(name="Synthesize", executor=synthesize)
    ]
)

print("\nТЕСТ 1: Факт из базы (должен найти в базе + проверить в интернете)")
request1 = ResearchTopic(topic="Инстасамка любимое блюдо")
result1 = workflow_3.run(input=request1)
if result1 and result1.content:
    display(Markdown(result1.content))

print("\nТЕСТ 2: Факт вне базы (только интернет)")
request2 = ResearchTopic(topic="Когда у Инстасамки была свадьба и есть ли у нее дети ?")
result2 = workflow_3.run(input=request2)
if result2 and result2.content:
    display(Markdown(result2.content))

print("\nТЕСТ 3: Противоречие (база vs интернет)")
request3 = ResearchTopic(topic="Когда родилась Инстасамка?")
result3 = workflow_3.run(input=request3)
if result3 and result3.content:
    display(Markdown(result3.content))


ТЕСТ 1: Факт из базы (должен найти в базе + проверить в интернете)


**Итог:** Согласно внутренней базе, любимое блюдо Инстасамки (Дарьи Якубович) — блины «Маникена». В то же время в публичных постах и интервью она не раз упоминала, что любит пиццу и суши, а в своём Instagram делилась рецептом пиццы.

**Источники:** оба (Internal DB; интернет — примеры: vk.com/wall-134965725_274829, instagram.com/reel/C-si1PHtt75/, youtube.com/shorts/M0n5Xut0E3o)

**Примечания:** Противоречие: внутренние данные указывают на «блины Маникена» как на любимое блюдо, тогда как публичные публикации и видео демонстрируют привязанность к пицце и суши и содержат рецепт пиццы. Разница может объясняться: внутренний «секретный» факт — личное или устаревшее утверждение; публичные посты отражают текущие предпочтения или любимые блюда в повседневности. Рекомендация: для окончательного подтверждения проверьте официальные источники Дарьи (официальный Instagram, интервью, прямые заявления) или запросите комментарий у её команды.


ТЕСТ 2: Факт вне базы (только интернет)


**Итог:** По публичным источникам Дарья Зотеева (Инстасамка) официально выходила замуж за рэпера/продюсера Moneyken (Олег Еропкин). Свадьба прошла приватно в ноябре 2021 года. В открытых источниках нет подтверждённых сведений о наличии у пары детей.

**Источники:** интернет

**Примечания:** База данных пуста — информация взята из публичных интернет-источников: статья 72.ru (декабрь 2022), страница в Википедии и соответствующий Instagram-пост. Противоречий между базой и интернетом нет. Рекомендую проверять официальные аккаунты и крупные СМИ для возможных обновлений о семейном статусе и детях.


ТЕСТ 3: Противоречие (база vs интернет)


**Итог:** По внутренней базе данных Инстасамка (Дарья Зотеева) родилась 2 июля 2000 года. Однако открытые интернет-источники (включая англоязычную статью Wikipedia и страницу last.fm) указывают дату 11 мая 2000 года.

**Источники:** оба (внутренняя база данных и интернет: Wikipedia, Last.fm)

**Примечания:** Противоречие: внутренняя база — 2 июля 2000; интернет-источники — 11 мая 2000. Разница может быть связана с ошибкой при вводе данных, разными версиями биографии в сети или обновлениями публичных профилей. Рекомендую проверить официальные и первоисточники: верифицированные соцсети артистки, официальные интервью, пресс-релизы или данные правообладателей/агентства для окончательного подтверждения.